In [11]:
# INST414 Final Project Sprint 2

## Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

# select columns to read and keep
cols_to_keep = ['Data_Value', 'Race/Ethnicity', 'Age(months)', 'Sex', 'LocationDesc', 'LocationAbbr', 'YearStart', 'YearEnd', 'Low_Confidence_Limit', 'High_Confidence_Limit', 'Sample_Size', 'GeoLocation']
df = pd.read_csv("/Users/virginialee/Downloads/WIC_data.csv", usecols=cols_to_keep)

df.head()
df.shape

(12852, 12)

In [12]:
# rename columns to be more intuitive
df.rename(columns={
    'Data_Value': 'pct_overweight',
    'Race/Ethnicity': 'race',
    'Age(months)': 'age_months'
}, inplace=True)
df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,Sex,race,GeoLocation
0,2008,2008,AL,Alabama,15.3,14.7,15.8,"18,219",24 - 35,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
1,2008,2008,AL,Alabama,14.9,14.4,15.5,"14,796",36 - 47,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
2,2008,2008,AL,Alabama,16.4,15.6,17.1,"10,272",48 - 59,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,NaN,American Indian/Alaska Native,"(32.84057112200048, -86.63186076199969)"
4,2008,2008,AL,Alabama,8.8,5.4,12.2,273,NaN,NaN,Asian/Pacific Islander,"(32.84057112200048, -86.63186076199969)"


In [13]:
# checking missing values
missing_value = df.isna().sum()
print(missing_value)

YearStart                    0
YearEnd                      0
LocationAbbr                 0
LocationDesc                 0
pct_overweight             379
Low_Confidence_Limit       379
High_Confidence_Limit      379
Sample_Size                379
age_months                9072
Sex                      10584
race                      7182
GeoLocation                  0
dtype: int64


In [6]:
# calculate % missing for key columns
379/12852 # 2.9% missing for data_value
379/12852 # 2.9% missing for low_confidence_limit
379/12852 # 2.9% missing for high_confidence_limit
379/12852 # 2.9% missing for sample_size
7182/12852 # 55.88% missing for race/ethnicity
9072/12852 # 70.59% missing for age
10584/12852 # 82.35% missing for sex

0.8235294117647058

In [14]:
# dropping missing values and unnecessary columns
clean_df = df.dropna(subset=['pct_overweight', 'Low_Confidence_Limit', 'High_Confidence_Limit', 'Sample_Size'])
clean_df = clean_df.drop(columns=['Sex'])
clean_df.head()
clean_df.shape

(12473, 11)

In [27]:
# filter race by American Indian/Alaska Native and Non-Hispanic White
df_race = clean_df[clean_df['race'].isin(['American Indian/Alaska Native', 'Non-Hispanic White'])]
df_race.shape
df_race.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,American Indian/Alaska Native,"(32.84057112200048, -86.63186076199969)"
9,2008,2008,AL,Alabama,15.8,15.2,16.3,"17,833",NaN,Non-Hispanic White,"(32.84057112200048, -86.63186076199969)"
14,2008,2008,AK,Alaska,23.7,22.2,25.2,"3,190",NaN,American Indian/Alaska Native,"(64.84507995700051, -147.72205903599973)"
20,2008,2008,AK,Alaska,15.5,14.3,16.7,"3,540",NaN,Non-Hispanic White,"(64.84507995700051, -147.72205903599973)"
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,"1,101",NaN,American Indian/Alaska Native,"(34.865970280000454, -111.76381127699972)"


In [25]:
# duplicates in data set
print(df_race.duplicated().value_counts()) # how many rows are duplicates
df_race[df_race.duplicated(keep=False)] # displays duplicate rows
# drop duplicates
df_race = df_race.drop_duplicates()
df_race.shape

False    2021
Name: count, dtype: int64


(2021, 11)

In [31]:
# relabel values to be more intuitive
df_race = df_race.replace({
    'American_Indian': 'Indigenous',
    'Non-Hispanic White': 'White'
}, inplace=False)
df_race.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,Indigenous,"(32.84057112200048, -86.63186076199969)"
9,2008,2008,AL,Alabama,15.8,15.2,16.3,"17,833",NaN,White,"(32.84057112200048, -86.63186076199969)"
14,2008,2008,AK,Alaska,23.7,22.2,25.2,"3,190",NaN,Indigenous,"(64.84507995700051, -147.72205903599973)"
20,2008,2008,AK,Alaska,15.5,14.3,16.7,"3,540",NaN,White,"(64.84507995700051, -147.72205903599973)"
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,"1,101",NaN,Indigenous,"(34.865970280000454, -111.76381127699972)"
